<a href="https://colab.research.google.com/github/biamouras/alura_pyspark_challenge/blob/main/Notebooks/02_tratamento_dados_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insightplaces - Tratamento dos dados para ML

A Insightplaces, plataforma fantasia de imóveis da Alura, está com dificuldades para definir os valores do imóveis e gostaria de fazer de modo automatizado. Ainda, a empresa percebeu que o recomendador atual não está sendo efetivo para gerar novos cliques entre os usuários.

Nosso projeto será trabalhar com os dados disponibilizados de imóveis no Rio de Janeiro para ajudar a empresa a lidar com estes dois problemas usando regressão para a definição dos valores e a análise de agrupamentos através de Machine Learning. 

Este segundo projeto tem o foco de preparar os dados para os modelos de previsão a partir de dados já tratados.

## Carregando os dados

In [1]:
from os import environ, listdir

In [2]:
# instalar as dependências para utilizar Spark no Colab
instala = True

# verifica se já está instalado 
for f in listdir('/content'):
  if f.find('spark') != -1:
    instala = False
    break 

if instala:
  !apt-get update -qq
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null

  !wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
  !tar xf spark-3.1.2-bin-hadoop2.7.tgz
  !pip install -q findspark

environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

In [3]:
# iniciando o pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [4]:
# monta a conexão com o drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# biblioteca para abrir o arquivo compactado
import zipfile

In [25]:
# caminho do arquivo 
name = 'semana-2'
dir_dados = '/content/drive/MyDrive/Colab Notebooks/alura_challenge_insightplaces/dados/' + name + '/'

# descompressão do arquivo
zipfile.ZipFile(dir_dados + name + '.zip', 'r').extractall(dir_dados)
# identificação do arquivo descomprimido
files = listdir(dir_dados)
files 

['semana-2.zip', 'dataset_transformado_parquet']

In [30]:
# o zip contém uma pasta
dir_dados_originais = dir_dados + files[1]
# verifica os arquivos dentro da pasta comprimida
files = listdir(dir_dados_originais)
files

['_SUCCESS',
 '._SUCCESS.crc',
 'part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet',
 '.part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet.crc']

In [32]:
# leitura do parquet
dados = spark.read.parquet(dir_dados_originais + '/' + files[2])
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



## Seleção de features